In [1]:
from pyspark.sql.functions import *
import pandas as pd
import numpy as np
import datetime
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, count, when
from pyspark.sql import Row
from pyspark.sql.functions import to_timestamp, broadcast

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 3, Finished, Available)

In [2]:
# path = "abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/8dfa5c43-7d50-4754-9b90-3c237db830de/Files/"
# file_name = "112017.csv"

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 4, Finished, Available)

In [3]:
# Load data from Order Partition
year = 2015
month = 1
path = 'abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/8dfa5c43-7d50-4754-9b90-3c237db830de/Files'
file_name = f'/dataset/year={year}/month={month}'

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 5, Finished, Available)

In [4]:
# from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#     .appName("Read CSV with PySpark") \
#     .getOrCreate()

# ISO-8859-1 UTF-8
# df = spark.read.format("csv") \
#         .option("header", "true") \
#         .option("encoding", "ISO-8859-1") \
#         .option("inferSchema", "true") \
#         .load(path + file_name + "/*.csv")

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 6, Finished, Available)

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Read CSV with PySpark") \
    .getOrCreate()



# Read the CSV file into a DataFrame
df = spark.read.format("csv") \
        .option("header", "true") \
        .option("encoding", "ISO-8859-1") \
        .option("inferSchema", "true") \
        .load("Files/DataCoSupplyChainDataset.csv")

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 7, Finished, Available)

In [6]:
df = df.filter(df["order date (DateOrders)"].like("1/%/2015%"))

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 8, Finished, Available)

## **Append to data source**

In [7]:
df_old = spark.read.format("csv") \
        .option("header", "true") \
        .option("encoding", "ISO-8859-1") \
        .option("inferSchema", "true") \
        .load("abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/8dfa5c43-7d50-4754-9b90-3c237db830de/Files/dataco/dataset_uploading")

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 9, Finished, Available)

In [8]:
print(df.count())
print(df_old.count())

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 10, Finished, Available)

5322
100


In [9]:
df_append = df.exceptAll(df_old)
df_append.count()

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 11, Finished, Available)

5285

In [10]:
df_append.write \
        .mode("append") \
        .format("csv") \
        .option("header", "true") \
        .save("Files/dataco/dataset_uploading")

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 12, Finished, Available)

# **DataCo Supply Chain Dataset**

In [11]:
def clean_column_names(df):
    for col in df.columns:
        new_col = col.strip().replace(" ", "_").replace("(", "").replace(")", "").lower()
        df = df.withColumnRenamed(col, new_col)
    return df

# Now let's clean the columns of your dataframe
df = clean_column_names(df)

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 13, Finished, Available)

## **Data Preprocessing**

In [12]:
null_counts = df.select([count(when(isnull(c), c)).alias(c) for c in df.columns])
null_counts.select([c for c in null_counts.columns if null_counts.collect()[0][c] > 0]).show()

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 14, Finished, Available)

+-------------+-------------------+
|order_zipcode|product_description|
+-------------+-------------------+
|         5322|               5322|
+-------------+-------------------+



In [13]:
# drop 2 cột có nhiều giá trị null nhất
df = df.drop('order_zipcode', 'product_description', 'customer_zipcode')

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 15, Finished, Available)

In [14]:
df = df.withColumnRenamed("type", "transaction_type") \
    .withColumnRenamed("order_date_dateorders", "order_date") \
    .withColumnRenamed("shipping_date_dateorders", "shipping_date")

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 16, Finished, Available)

In [15]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
df = df.withColumn("order_date", to_timestamp(df["order_date"], "MM/dd/yyyy HH:mm"))
df = df.withColumn("shipping_date", to_timestamp(df["shipping_date"], "MM/dd/yyyy HH:mm"))


StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 17, Finished, Available)

In [16]:
display(df.filter(col('customer_state').rlike('^\\d+$')))

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 18, Finished, Available)

SynapseWidget(Synapse.DataFrame, eeb09720-603e-4ffd-9f11-c0fcee10710f)

In [17]:
df = df.filter(~col('customer_state').rlike('^\\d+$')) # filter out state which is a number

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 19, Finished, Available)

In [18]:
df.count()

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 20, Finished, Available)

5322

## **Add 2 more columns**

In [19]:
df = df.withColumn("concat_destination_address", concat(col("order_city"), col("order_state"), col("order_country"), col("order_region"), col("market")))
df = df.withColumn("concat_customer_region", concat(col("customer_country"), col("customer_state"), col("customer_city"), col("customer_street")))

df = df.withColumnRenamed('product_card_id', 'product_id')

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 21, Finished, Available)

In [20]:
from pyspark.sql.functions import regexp_replace, ltrim
df = df.withColumn('customer_street', regexp_replace('customer_street', '[0-9]', ''))
df = df.withColumn('customer_street', ltrim(df['customer_street']))

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 22, Finished, Available)

# **Load and Overwrite into Bronze Layer**

In [21]:
df.write.format("delta").mode("overwrite").saveAsTable('LTT_BronzeLakehouse.cleaned_df')

StatementMeta(, 51630aa9-f448-4d3f-91e9-2000233d117f, 23, Finished, Available)